In [1]:
%%capture
!pip install -U spacy==2.3.7
!pip3 install pickle5

#@title ## Laden der Umgebung
#@markdown Wählen Sie in der Menüleiste unter "Laufzeit" den Punkt "Alle ausführen" oder drücken Sie STRG+F9
from IPython.display import clear_output 
from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
clear_output()

import os
import numpy as np
import sys
import pathlib

from spacy import displacy

src = "NLP-on-multilingual-coin-descriptions/"
storage = "temp_storage/"
ner_path = src + "cnt/trained_model/ner/"

if not os.path.exists(src):
  !wget -O file.zip 'https://docs.google.com/uc?export=download&id=1wIPNEVzFG8610LauJzCVc2Rk3W3UmDbj&confirm=t'
  !unzip file.zip
  os.remove(ner_path+"desktop.ini")
if not os.path.exists(storage):
  os.mkdir(storage)
  
if os.path.exists(ner_path+".ipynb_checkpoints"):
  os.rmdir(ner_path+".ipynb_checkpoints")
  


sys.path.append(src)
from cnt.model import load_ner_model_v2
from cnt.model import load_pipeline, predict_re_single_sentence

In [3]:
#@title
#@markdown Es kann eine Textdatei hochgeladen werden, diese sollte im folgenden Format sein:
#@markdown (Trennzeichen: ;;)
#@markdown <br/><br/> Design;;Design;;Design;; <br/>
#@markdown <br/> Ein Design kann dann über das Dropdown Menü ausgewählt werden.
#@markdown <br/> Eine weitere Möglichkeit ist es das Design im Textfeld einzugeben.


# Uploaders
uploader = widgets.FileUpload(
    description="Upload designs",
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)

# Editable fields
textfield = widgets.Text(
    value='',
    placeholder='Type something',
    description='Design:',
    disabled=False
)

# Buttons
button = widgets.Button(description="Upload")
predict_button = widgets.Button(description="Predict")
buttonds = widgets.Button(description="Delete Session")

# Output fields
output = widgets.Output()

# Non editable fields
text_language = widgets.HTML(
    value="<b>Select language: </b>"
)

widgets.HTML(
    value="<b>Select language: </b>"
)

# Dropdowns
drop = widgets.Dropdown(
    options=os.listdir(ner_path),
    disabled=False, continuous_update=True,layout={'width': 'max-content'}
)

drop_designs = widgets.Dropdown(
    options=[],
    disabled=False, continuous_update=True
)

# Radio buttons
output_radio = widgets.RadioButtons(
    options=[('Position + Class',0), ('Entity + Class',1),
             ('Full Sentence with prediction',2)],
    description='Output style',
    disabled=False
)


# Functions
def predict_on_drop(b):
    predict_img(storage+drop.value)
  

design_list = []

def on_button_clicked(b):


  for img in uploader.value:
    uploaded_file = uploader.value[img]
    for design in str(uploaded_file["content"])[2:-3].split(";;"):
      design_list.append(design)

  drop_designs.options = design_list
  uploader.value.clear()
  uploader._counter=0

# Accordion menu
accordion = widgets.Accordion(children=[VBox([uploader, button, drop_designs]), textfield])
accordion.set_title(0, 'Upload txt file')
accordion.set_title(1, 'Use text field')

def load_language(language):

  model_directory =  ner_path + drop.value+"/"

  if language == "english":
    model_name = "english_cno"

  elif language== "german":
    model_name = "german_cno"

  return load_ner_model_v2(model_directory, model_name)

# Predict function
def apply_nlp(b):
  


  MODEL = load_language(drop.value)

  if accordion.selected_index == 0:
        sentence = drop_designs.value
  
  elif accordion.selected_index == 1:
    sentence = textfield.value

  if output_radio.value == 0:
    prediction = MODEL.predict_single_sentence(sentence)
    #print(prediction)

  elif output_radio.value == 1:
    prediction = MODEL.predict_single_sentence_clear(sentence, as_doc=False)
    #print(prediction)
  
  elif output_radio.value == 2:
    prediction = MODEL.predict_single_sentence_clear(sentence, as_doc=True)

    colors = {'PERSON': 'mediumpurple','OBJECT': 'greenyellow', 'ANIMAL' : 'orange', 'PLANT': 'salmom', 'VERBS': 'skyblue'}
    options = {'ent': ['PERSON', 'OBJECT', 'ANIMAL', 'PLANT'], 'colors': colors}
    #prediction = displacy.render(prediction,style='ent', jupyter=True, options=options)
    
  with output:
    output.clear_output()
    print()
    display(sentence)
    if output_radio.value == 2:
      displacy.render(prediction,style='ent', jupyter=True, options=options)
    else:
      display(prediction)

def delete_sess(b):
  clear_output()



button.on_click(on_button_clicked)
predict_button.on_click(apply_nlp)


display(accordion)
display(HBox([text_language, drop, output_radio]))
display(predict_button)
display(output)



Accordion(children=(VBox(children=(FileUpload(value={}, description='Upload designs', multiple=True), Button(d…

Button(description='Predict', style=ButtonStyle())

Output()